In [ ]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from business_score import custom_score
from imblearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix 

In [ ]:

my_data = joblib.load('./models/data/test.joblib')
my_data.head()

In [ ]:
scaler = joblib.load('./models/scaler.model')

In [ ]:
explainer = joblib.load('./models/explainer.model')

In [ ]:
lgbm_model = joblib.load('./models/LGBMClassifier.model')



In [ ]:
df_test  = my_data.copy()

In [ ]:
df_test.head()

In [ ]:
test = df_test[df_test.columns[df_test.columns != "SK_ID_CURR"]]
test.shape

### Prediction de pret a partir de fichier test:
* Standardisation des données de test:

In [ ]:
test_scaler = scaler.transform(test)

* Prediction:

In [ ]:
y_pred = lgbm_model.predict(test_scaler)

In [ ]:
y_pred_proba = lgbm_model.predict_proba(test_scaler)
y_pred_proba

In [ ]:
proba_df = pd.DataFrame(y_pred_proba,columns = ['Class_0','Class_1'], index = df_test['SK_ID_CURR'])
proba_df

In [ ]:
proba_df['y_pred'] = y_pred
proba_df.shape

In [ ]:
proba_df['y_pred'].value_counts()

In [ ]:
shap_values_test = explainer.shap_values(test_scaler)

### Features importance:

In [ ]:
shap.initjs()
shap.summary_plot(shap_values_test[0],
                  features = test,
                  feature_names = test.columns,
                  plot_size = (10,8),
                  show = False

                  )
plt.title("Interprétation Globale (shap-values)",fontsize = 14)
plt.tight_layout()
plt.show()

### Features local:

In [ ]:
explainer.expected_value

In [ ]:
shap_values_test[1]

In [ ]:
len(shap_values_test[0][1])

In [ ]:
instance_idx = 0
test.iloc[instance_idx, :]

In [ ]:
id_client  = df_test[df_test['SK_ID_CURR']== 456202]
instance_idx =  df_test[df_test['SK_ID_CURR']== 456202].index
shap.initjs()  
shap.force_plot(explainer.expected_value[1], shap_values_test[1][instance_idx,:],
                 id_client.drop(columns=["SK_ID_CURR"],axis = 1))

In [ ]:
id_client  = df_test[df_test['SK_ID_CURR']== 456202]
id_client.index

#### Comparaison avec d'autres clients:

In [ ]:
group_client = df_test.merge(proba_df, how='left',on = 'SK_ID_CURR')
group_client.head()

In [ ]:
group_client['y_pred'] = group_client['y_pred'].map({0: 'Accordé', 1: 'Refusé'})


In [ ]:
client =group_client[group_client['SK_ID_CURR'] == 100001]

In [ ]:
path =  "./models/data/df.joblib"
# Enregistrez le modèle
joblib.dump(group_client, path)